In [1]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from datetime import datetime

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Load dataset

In [3]:
df = pd.read_csv('FULL_NEWS.csv')

In [4]:
df_new = df.reindex(columns=['_id', 'news_title', 'first_crypto', 'second_crypto', 'third_crypto', 'source', 'news_link', 'publication_date', 'pos_votes', 'neg_votes', 'imp_votes', 'com_votes', 'like_votes', 'dislike_votes', 'lol_votes', 'save_votes'])

In [5]:
df_new.head()

,_id,news_title,first_crypto,second_crypto,third_crypto,source,news_link,publication_date,pos_votes,neg_votes,imp_votes,com_votes,like_votes,dislike_votes,lol_votes,save_votes
0,14630691,"This Week on Crypto Twitter: Elon Musk's Diamond Hands, Tom Brady Responds to Vitalik",NaN,NaN,NaN,decrypt.co,https://cryptopanic.com/news/14630691/This-Week-on-Crypto-Twitter-Elon-Musks-Diamond-Hands-Tom-Brady-Responds-to-Vitalik,2022.03.20,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14630675,Inside the blockchain developers’ mind: How to avoid development hell,NaN,NaN,NaN,cointelegraph.com,https://cryptopanic.com/news/14630675/Inside-the-blockchain-developers-mind-How-to-avoid-development-hell,2022.03.20,NaN,NaN,1.0,1.0,1.0,NaN,NaN,1.0
2,14630674,Hoskinson Promises Solana-Like Performance for Cardano,ADA,NaN,NaN,beincrypto.com,https://cryptopanic.com/news/14630674/Hoskinson-Promises-Solana-Like-Performance-for-Cardano,2022.03.20,2.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN
3,14630679,"Bitcoin (BTC) Supply on Crypto Exchanges Drops to Three-Year Lows, According to Analytics Firm",BTC,ETH,NaN,dailyhodl.com,https://cryptopanic.com/news/14630679/Bitcoin-BTC-Supply-on-Crypto-Exchanges-Drops-to-Three-Year-Lows-According-to-Analytics-Firm,2022.03.20,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14630608,Ethereum's Vitalik Buterin Comes with New Sharding Proposal: What is EIP-4844?,ETH,NaN,NaN,u.today,https://cryptopanic.com/news/14630608/Ethereums-Vitalik-Buterin-Comes-with-New-Sharding-Proposal-What-is-EIP-4844,2022.03.20,2.0,NaN,1.0,1.0,2.0,NaN,1.0,NaN


In [6]:
df_new = df_new.sort_values('publication_date', ascending=True)

In [7]:
df_new.isna().sum()

_id                     0
news_title              0
first_crypto         3328
second_crypto        7937
third_crypto         9305
source                572
news_link               0
publication_date        0
pos_votes            8378
neg_votes           10033
imp_votes            8872
com_votes            9941
like_votes           8245
dislike_votes       10116
lol_votes            9966
save_votes           9828
dtype: int64

# Replace all NaNs in votes with 0

In [8]:
reac = ['pos_votes', 'neg_votes', 'imp_votes', 'com_votes', 'like_votes', 'dislike_votes', 'lol_votes', 'save_votes']

for i in reac:
    df_new[i] = df_new[i].fillna(0)

# Remove third_crypto, too many NaNs

In [9]:
df_new = df_new.drop(columns='third_crypto') 

# Remove news_link, doesn't matter

In [10]:
df_new = df_new.drop(columns='news_link') 

# Fiil in source with popular value

In [11]:
mode = df_new['source'].mode()
df_new['source'] = df_new['source'].fillna(mode[0])
df_new['source'].isna().sum()

0

## Remove @ 

In [12]:
df_new['source'] = df_new['source'].apply(lambda x: x.replace("@", ''))

# Remove stopwords

Gensim stopwords and punctuations

In [13]:
df_new['news_title'] = df_new['news_title'].apply(lambda x: remove_stopwords(x))

In [14]:
all_stopwords_gensim = STOPWORDS.union(set([',', '.', ':', "'", '…', '?', '@', '#', '!', '’']))

df_new['news_title'] = df_new['news_title'].apply(lambda x: word_tokenize(x))
df_new['news_title'] = df_new['news_title'].apply(lambda x: [word for word in x if not word in all_stopwords_gensim])

NLTK russian and french stopwords

In [15]:
russian_stopwords = stopwords.words('russian')
df_new['news_title'] = df_new['news_title'].apply(lambda x: [word for word in x if not word in russian_stopwords])

french_stopwords = stopwords.words('french')
df_new['news_title'] = df_new['news_title'].apply(lambda x: [word for word in x if not word in french_stopwords])

In [16]:
df_new['news_title'] = df_new['news_title'].apply(lambda x: (" ").join(x))

In [17]:
df_new.head()

,_id,news_title,first_crypto,second_crypto,source,publication_date,pos_votes,neg_votes,imp_votes,com_votes,like_votes,dislike_votes,lol_votes,save_votes
882,14604850,We welcome Algolaunch Algorand ecosystem AlgoLaunch recipient IDO Launchpad SupaGrant wil…,ALGO,NaN,cointelegraph.com,2022.03.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
757,14602502,Ripple Distributes 1 Billion XRP In Developer Grants As Chief Developer Resigns,XRP,NaN,bitcoinist.com,2022.03.17,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
756,14602447,Treasure_DAO integrating Chainlink Price Feeds help secure pricing TroveByTreasure forthcoming…,LINK,NaN,smart_contract,2022.03.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
755,14608066,The Hedera22 Hackathon ️ 6 weeks 16 challenges $ 400k+ prizes Hello smart contracts Hello future,HBAR,NaN,hedera,2022.03.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
754,14609384,Let BUIDL MinaCommunity The Mina Ecosystem raised $ 92 Million dollars raise led FTX_Official a…,MINA,MINA,minaprotocol,2022.03.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# One Hot Encoding (first_crypto, seccond_crypto)

In [18]:
df_new['first_crypto'].value_counts()

BTC       2107
ETH        637
BNB        480
LINK       293
HT         228
SAND       212
AVAX       201
ADA        201
MCO        194
ALGO       193
SOL        192
MATIC      152
HBAR       139
UST        134
NEXO       126
VET        117
CAKE       102
RUNE        96
SHIB        96
DOT         94
TRX         93
DOGE        91
XRP         91
APE         85
FIL         83
MANA        81
FTT         81
NEAR        81
LUNA        73
LTC         68
MIOTA       66
XTZ         65
BTT         59
ERD         59
ZIL         59
USDT        54
AXS         52
LEO         51
ICP         50
GALA        49
XLM         48
GST         47
KDA         44
CELO        40
LRC         37
EXRD        35
TUSD        34
UNI         31
FLOW        31
MINA        30
GRT         30
ATOM        27
ENJ         26
THETA       24
USDC        23
XMR         22
CHZ         21
KLAY        20
HOT         20
HNT         19
MKR         18
AAVE        18
CEL         15
KSM         14
SPELL       14
MC_         13
FXS       

In [19]:
df_new['second_crypto'].value_counts()


ETHDOWN     451
ETH         318
LUNA        294
HPT         228
CRO         194
WAVAX       169
SOL         118
VTHO        115
DOT          92
HOLY         80
FIL12        79
SOLINK       72
ADA          70
EGLD         61
FLOW         59
BTTOLD       59
GMT          53
WNEAR        51
ICP          51
GZIL         49
TOWN         48
SHIB         47
USDT         41
DOGE         36
XRD          35
BCH          35
CELO         34
AXS          30
AVAX         30
NEAR         30
LRN          29
MATIC        29
MINA         26
APE          26
XRP          25
TFUEL        24
EFI          24
USDC         23
KLAY         21
LEASH        20
HOT          20
WFLOW        17
PFL          17
BETH         16
UNI          15
UST          15
XAUT         14
MIM          14
CEL          14
BTC          14
FRAX         13
XCN          12
LDO          12
AAVE         12
MKR          11
OSMO         10
BNB          10
DAI           9
NEXO          8
ALGO          7
XDC           7
LEND          6
KSM     

In [21]:
# Dummies for first_crypto and second_crypto one hot encoding
df_new = pd.get_dummies(df_new, columns=["first_crypto", "second_crypto"], prefix=["first", "second"], dummy_na=True)

# Load quote data

In [22]:
df_crypto = pd.read_excel('BTCUSDT_1d_17 Mar, 2022_20 Apr, 2022.xlsx')

In [23]:
df_crypto.rename(columns={'dateTime': 'publication_date'}, inplace=True)

In [24]:
df_crypto.head()

,publication_date,open,high,low,close,volume
0,17.03.22 00:00:00,41114.01,41478.82,40500.00,40917.90,37189.38087
1,18.03.22 00:00:00,40917.89,42325.02,40135.04,41757.51,45408.00969
2,19.03.22 00:00:00,41757.51,42400.00,41499.29,42201.13,29067.18108
3,20.03.22 00:00:00,42201.13,42296.26,40911.00,41262.11,30653.33468
4,21.03.22 00:00:00,41262.11,41544.22,40467.94,41002.25,39426.24877


In [25]:
df_crypto['publication_date'] = df_crypto['publication_date'].apply(lambda x: datetime.strptime(x, "%d.%m.%y %H:%M:%S").strftime('%Y.%m.%d'))

In [26]:
full_dataset = df_new.merge(df_crypto, on='publication_date')

In [27]:
full_dataset.head()

,_id,news_title,source,publication_date,pos_votes,neg_votes,imp_votes,com_votes,like_votes,dislike_votes,lol_votes,save_votes,first_AAVE,first_ADA,first_ALGO,first_AMP,first_ANC,first_APE,first_AR,first_ATOM,first_AVAX,first_AXS,first_BAT,first_BCH,first_BCHABC,first_BIT,first_BNB,first_BTC,first_BTT,first_BUSD,first_CAKE,first_CEL,first_CELO,first_CHZ,first_COMP,first_CRO,first_CVX,first_CVXCRV,first_DAI,first_DCR,first_DFI,first_DOGE,first_DOT,first_EGLD,first_ENJ,first_EOS,first_ERD,first_ETC,first_ETH,first_EXRD,first_FIL,first_FLOW,first_FRAX,first_FTM,first_FTT,first_FXS,first_GALA,first_GMT,first_GNO,first_GNT,first_GRT,first_GST,first_GT,first_HBAR,first_HEART,first_HNT,first_HOT,first_HT,first_ICP,first_ION,first_JEWEL,first_JUNO,first_KCS,first_KDA,first_KLAY,first_KMD,first_KSM,first_LEND,first_LEO,first_LINK,first_LRC,first_LTC,first_LUNA,first_MANA,first_MATIC,first_MCO,first_MC_,first_MINA,first_MIOTA,first_MKR,first_NEAR,first_NEO,first_NEXO,first_OMG,first_OMI,first_QNT,first_RUNE,first_SAND,first_SHIB,first_SNX,first_SOL,first_SPELL,first_STETH,first_STX,first_SUSHI,first_THETA,first_TRX,first_TUSD,first_UNI,first_USDC,first_USDT,first_UST,first_VET,first_XCN,first_XDCE,first_XEC,first_XIDO,first_XLM,first_XMR,first_XRP,first_XTZ,first_ZEC,first_ZIL,first_nan,second_AAVE,second_ADA,second_ALGO,second_APE,second_AR,second_ATOM,second_AVAX,second_AXS,second_BCH,second_BETH,second_BNB,second_BTC,second_BTT,second_BTTOLD,second_BUSD,second_CAKE,second_CEL,second_CELO,second_CHZ,second_CRO,second_CVX,second_DAI,second_DFI,second_DFKTEARS,second_DOGE,second_DOT,second_DTSLA,second_EFI,second_EGLD,second_EOS,second_ETC,second_ETH,second_ETHDOWN,second_FIL,second_FIL12,second_FLOW,second_FRAX,second_FTM,second_FTT,second_GAS,second_GMT,second_GRT,second_GZIL,second_HBAR,second_HNT,second_HOLY,second_HOT,second_HPT,second_ICP,second_KCS,second_KLAY,second_KSM,second_LDO,second_LEASH,second_LEND,second_LINK,second_LRC,second_LRN,second_LTC,second_LUNA,second_MANA,second_MATIC,second_MIM,second_MINA,second_MKR,second_NEAR,second_NEXO,second_OKB,second_ONE,second_OSMO,second_PFL,second_QNT,second_RUNE,second_SAI,second_SAND,second_SHIB,second_SOL,second_SOLINK,second_STETH,second_STX,second_SUSHI,second_TFUEL,second_THETA,second_TOWN,second_TRX,second_TUSD,second_UNI,second_USDC,second_USDT,second_UST,second_VET,second_VTHO,second_WAVAX,second_WBTC,second_WFLOW,second_WNEAR,second_WOMI,second_XAUT,second_XCN,second_XDC,second_XEC,second_XLM,second_XMR,second_XRD,second_XRP,second_XTZ,second_ZIL,second_nan,open,high,low,close,volume
0,14604850,We welcome Algolaunch Algorand ecosystem AlgoLaunch recipient IDO Launchpad SupaGrant wil…,cointelegraph.com,2022.03.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,41114.01,41478.82,40500.0,40917.9,37189.38087
1,14602502,Ripple Distributes 1 Billion XRP In Developer Grants As Chief Developer Resigns,bitcoinist.com,2022.03.17,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,41114.01,41478.82,40500.0,40917.9,37189.38087
2,14602447,Treasure_DAO integrating Chainlink Price Feeds help secure pricing TroveByTreasure forthcoming…,smart_contract,2022.03.17,

In [28]:
full_dataset.to_csv('cryptoset.csv', sep=';', encoding='utf-8-sig')